In [45]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader


In [46]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [47]:
train_data = datasets.ImageFolder(
    root="/content/drive/MyDrive/MRI_ANALYSIS/Training",
    transform=transform
)

test_data = datasets.ImageFolder(
    root="/content/drive/MyDrive/MRI_ANALYSIS/Testing",
    transform=transform
)


In [48]:
train_loader = DataLoader(train_data,batch_size=8,shuffle=True)
test_loader = DataLoader(test_data,batch_size=8)

In [49]:
class MRINet(nn.Module):
  def __init__(self):
    super().__init__()
    self.base = models.resnet18(pretrained=True)
    self.base.fc = nn.Linear(self.base.fc.in_features,4)
  def forward(self,x):
    return self.base(x)

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MRINet().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [51]:
!pip install tqdm


In [52]:
from tqdm import tqdm
def train():
  model.train()
  total_loss = 0
  for images,labels in tqdm(train_loader, desc="Training", leave=False):
    images,labels = images.to(device),labels.to(device)
    optimizer.zero_grad()
    output = model(images)
    loss = loss_fn(output,labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  return total_loss/len(train_loader)


In [53]:
def evaluate():
  model.eval()
  correct,total = 0,0
  with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating", leave=False):
      images,labels = images.to(device),labels.to(device)
      preds = torch.argmax(model(images),dim=1)
      correct += (preds==labels).sum().item()
      total += labels.size(0)
  return correct/total

In [54]:
for epoch in tqdm(range(1, 30), desc="Epochs"):
  avg_loss = train()
  acc = evaluate()
  print(f"Epoch: {epoch}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}")

Training: 100%|██████████| 359/359 [00:46<00:00,  8.07it/s]
                                                           
Epochs:   3%|▎         | 1/29 [00:50<23:42, 50.79s/it]

Epoch: 1, Loss: 0.8562, Accuracy: 0.3503



Training: 100%|█████████▉| 358/359 [00:42<00:00,  9.23it/s]
                                                           
Epochs:   7%|▋         | 2/29 [01:37<21:39, 48.13s/it]

Epoch: 2, Loss: 0.6308, Accuracy: 0.4975



Training: 100%|██████████| 359/359 [00:41<00:00,  7.75it/s]
                                                           
Epochs:  10%|█         | 3/29 [02:23<20:27, 47.22s/it]

...

Epoch: 27, Loss: 0.1076, Accuracy: 0.7411



Training: 100%|██████████| 359/359 [00:47<00:00,  8.25it/s]
                                                           
Epochs:  97%|█████████▋| 28/29 [22:43<00:51, 51.86s/it]

Epoch: 28, Loss: 0.0999, Accuracy: 0.6675



Training: 100%|██████████| 359/359 [00:48<00:00,  8.58it/s]
                                                           
Epochs: 100%|██████████| 29/29 [23:36<00:00, 48.85s/it]

Epoch: 29, Loss: 0.0864, Accuracy: 0.7030


In [43]:
torch.save(model.state_dict(), "mri_classifier.pth")

In [44]:
from google.colab import files
files.download("mri_classifier.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>